# Extended RNA-Seq Analysis Training Demo

## Overview

This tutorial workflow uses the full dataset from Mittenbühler MJ et al., project.

The tutorial repeats the short tutorial, but with the full fastq files and includes some extra steps, such as how to access SRA metadata using Athena, a more powerful and fast step compared to sra-tools.

Full fastq files can be rather large, and so the downloading, extracting, and analysis of them means this tutorial can take over 13 hours to run the code fully using ml.m5.4xlarge instance.

All outputs used in the DEG tutorial were created using this extended full dataset tutorial workflow.

![RNA-Seq workflow](../images/rnaseq-workflow.png)

## STEP 1: Install Miniforge

First install Miniforge.

In [1]:
# Download Miniforge or Mambaforge (you can use either based on preference)
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh

# Install Miniforge (or Mambaforge) - no need to install conda since mamba will be available immediately
!bash Miniforge3-$(uname)-$(uname -m).sh -b -u -p $HOME/miniforge
!date +"%T"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 86.0M  100 86.0M    0     0   124M      0 --:--:-- --:--:-- --:--:--  124M
PREFIX=/home/ec2-user/miniforge

Transaction

  Prefix: /home/ec2-user/miniforge/envs/_virtual_specs_checks

  All requested packages already installed

Dry run. Not executing the transaction.
Unpacking payload ...
Extracting _libgcc_mutex-0.1-conda_forge.tar.bz2
Extracting ca-certificates-2024.8.30-hbcca054_0.conda
Extracting ld_impl_linux-64-2.40-hf3520f5_7.conda
Extracting pybind11-abi-4-hd8ed1ab_3.tar.bz2
Extracting python_abi-3.12-5_cp312.conda
Extracting tzdata-2024a-h8827d51_1.conda
Extracting libgomp-14.1.0-h77fa898_1.conda
Extracting _openmp_mutex-4.5-2_gnu.tar.bz2
Extracting libgcc-14.1.0

Next, using mambaforge and bioconda, install the tools that will be used in this tutorial.

In [2]:
# Update PATH to point to the Miniforge (or Mambaforge) bin files
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/miniforge/bin"

#now we can easily use 'mamba' command to install software 
!mamba install -y -c conda-forge -c bioconda trimmomatic fastqc multiqc sql-magic entrez-direct gffread parallel-fastq-dump sra-tools sql-magic pyathena samtools star rsem entrez-direct subread pigz -y


Looking for: ['trimmomatic', 'fastqc', 'multiqc', 'sql-magic', 'entrez-direct', 'gffread', 'parallel-fastq-dump', 'sra-tools', 'sql-magic', 'pyathena', 'samtools', 'star', 'rsem', 'entrez-direct', 'subread', 'pigz']

warning  libmamba Cache file "/home/ec2-user/anaconda3/pkgs/cache/497deca9.json" was modified by another program
warning  libmamba Cache file "/home/ec2-user/anaconda3/pkgs/cache/09cdf8bf.json" was modified by another program
[+] 0.0s
[+] 0.1s
bioconda/linux-64 (check zst) ━━━╸━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.1sbioconda/linux-64 (check zst)                       Checked  0.2s
[+] 0.0s
bioconda/noarch (check zst)                        Checked  0.0s
warning  libmamba Could not parse state file: Could not load cache state: [json.exception.type_error.302] type must be string, but is null
[+] 0.0s
[+] 0.1s
https://aws-ml-conda.s3.us-west-2.amazonaws.com/.. ━━━━━━━╸━━━━━━━   0.0 B  0.1shttps://aws-ml-conda.s3.us-west-2.amazonaws.com/..  0.2s
warning  libmamba Could 

## STEP 2: Setup Environment

Create a set of directories in the sra-data-athena to store the reads, reference sequence files, and output files. Notice that first we remove the `data` directory to clean up files from Tutorial_1

In [3]:
!cd $HOMEDIR
!echo $PWD
!mkdir -p data
!mkdir -p data/trunc_rawfastq
!mkdir -p data/trimmed
!mkdir -p data/fastqc
!mkdir -p data/fastqc_samples/
!mkdir -p data/reference
!mkdir -p data/aligned_bam
!mkdir -p data/rsem_reference/mouse_rsem_reference
!mkdir -p data/rsem_output
!mkdir -p data/reference/STAR_index

/home/ec2-user/SageMaker


Set # THREADS depending on your VM size

In [4]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
THREADS = max(1, num_cores - 1)

print("Number of threads:", THREADS)
os.environ["THREADS"] = str(THREADS)

Number of threads: 15


## STEP 3: Downloading relevant FASTQ files using SRA Tools



Next we will need to download the relevant fastq files.

Because these files can be large, the process of downloading and extracting fastq files can be quite lengthy.

We will be downloading the sample runs from this project using SRA tools, downloading from the NCBI's SRA (Sequence Run Archives).

However, first we need to find the associated accession numbers in order to download.

### STEP 3.1: Finding run accession numbers.


The SRA stores sequence data in terms of runs, (SRR stands for Sequence Read Run). To download runs, we will need the accession ID for each run we wish to download.

The Mittenbühler MJ et al., project contains 8 runs. To make it easier, these are the run IDs associated with this project:

- SRR21972730
- SRR21972729
- SRR21972728
- SRR21972727
- SRR21972725
- SRR21972724
- SRR21972723
- SRR21972726

In this case, all these runs belong to the Bioproject PRJNA892075.

Sequence run experiments can be searched for using the SRA database on the NCBI website; and article-specific sample run information can be found in the supplementary section of that article.

For instance, here, the the authors posted a link to the sequence data GSE (Gene Series number), GSE164210. This leads to the appropriate 'Gene Expression Omnibus' page where, among other useful files and information, the relevant SRA database link can be found.

Once the accession numbers are located, one can make a text file containing the list of accession IDs however they like.

Once again, to make things easier, we have made a .txt with these IDs that you can simply download here:

In [5]:
!esearch -db sra -query "PRJNA625528" | efetch -format runinfo | cut -d',' -f1 | tail -n +2 > all_accs.txt
!grep -E "SRR11550221|SRR11550223|SRR11550225|SRR11550227|SRR11550229|SRR11550231" all_accs.txt > accs.txt
!cat accs.txt

SRR11550221
SRR11550223
SRR11550225
SRR11550227
SRR11550229
SRR11550231


### STEP 3.2 Finding run accession numbers using Athena (Optional)

Athena is a serverless query engine that allows you to analyze data stored in Amazon S3 using standard SQL. It offers several advantages over traditional SRA tools, making it a more efficient and scalable solution for large-scale RNA-seq data analysis.

Using Athena to access metadata is optional, but allows you to query large SRA metadata directly from AWS without needing to download and process files locally, making it faster and more scalable.

In [ ]:
from pyathena import connect
import pandas as pd

# Use the correct argument name: s3_staging_dir
conn = connect(s3_staging_dir='s3://sra-data-athena/', region_name='us-east-1')

In [ ]:
import boto3

# Initialize the Glue client
glue_client = boto3.client('glue', region_name='us-east-1')

# Run the crawler
crawler_name = 'sra_crawler'  # Use your crawler's name
glue_client.start_crawler(Name=crawler_name)

print(f"Crawler {crawler_name} started.")

In [ ]:
query = """
SELECT *
FROM AwsDataCatalog.srametadata.metadata
WHERE bioproject = 'PRJNA892075'
"""
df = pd.read_sql(query, conn)
df


In [ ]:
#write the SRR column to a text file
with open('accs.txt', 'w') as f:
    accs = df['acc'].to_string(header=False, index=False)
    f.write(accs)
    
#print the text file
!cat accs.txt

### STEP 3.3 Using the SRA-toolkit for a single sample.

The code snippet demonstrates how to download and preprocess single SRA data using the prefetch and fasterq-dump commands. First, prefetch downloads the specified SRA file and then, fasterq-dump converts the SRA file into paired-end FASTQ files. Finally, the generated FASTQ files are compressed using pigz to save space.

In [6]:
# Example usage for SRA download:
!prefetch SRR11550221 -O data/raw_fastq -f yes

2024-10-09T05:56:00 prefetch.3.1.1: 1) Resolving 'SRR11550221'...
2024-10-09T05:56:00 prefetch.3.1.1: Current preference is set to retrieve SRA Normalized Format files with full base quality scores
2024-10-09T05:56:00 prefetch.3.1.1: 1) Downloading 'SRR11550221'...
2024-10-09T05:56:00 prefetch.3.1.1:  SRA Normalized Format file is being retrieved
2024-10-09T05:56:00 prefetch.3.1.1:  Downloading via HTTPS...
2024-10-09T05:56:54 prefetch.3.1.1:  HTTPS download succeed
2024-10-09T05:56:56 prefetch.3.1.1:  'SRR11550221' is valid: 1357522631 bytes were streamed from 1357514436
2024-10-09T05:56:56 prefetch.3.1.1: 1) 'SRR11550221' was downloaded successfully
2024-10-09T05:56:56 prefetch.3.1.1: 'SRR11550221' has 0 dependencies


In [7]:
#convert sra to fastq
!fasterq-dump data/raw_fastq/SRR11550221 -f -O data/raw_fastq/ -e $THREADS
#compress fastq to fastq.gz to save space
!pigz -p $THREADS data/raw_fastq/SRR11550221.fastq

spots read      : 46,053,552
reads read      : 92,107,104
reads written   : 46,053,552
reads 0-length  : 46,053,552


### STEP 3.4 Downloading multiple files using the SRA-toolkit.

The code uses prefetch to download multiple SRA files in parallel. It reads the list of SRR IDs from accs.txt, uses xargs to execute prefetch for each ID, and specifies the output directory and the -f option to create FASTQ files in the same directory as the SRA files. To speed up the download the code uses -P $THREADS option allowing parallel execution using the specified number of threads.

In [8]:
!cat accs.txt | xargs -P $THREADS -I {} prefetch {} -O data/raw_fastq -f yes

2024-10-09T06:01:15 prefetch.3.1.1: 1) Resolving 'SRR11550227'...
2024-10-09T06:01:15 prefetch.3.1.1: 1) Resolving 'SRR11550225'...
2024-10-09T06:01:15 prefetch.3.1.1: 1) Resolving 'SRR11550223'...
2024-10-09T06:01:15 prefetch.3.1.1: 1) Resolving 'SRR11550221'...
2024-10-09T06:01:15 prefetch.3.1.1: 1) Resolving 'SRR11550229'...
2024-10-09T06:01:15 prefetch.3.1.1: 1) Resolving 'SRR11550231'...
2024-10-09T06:01:15 prefetch.3.1.1: Current preference is set to retrieve SRA Normalized Format files with full base quality scores
2024-10-09T06:01:15 prefetch.3.1.1: Current preference is set to retrieve SRA Normalized Format files with full base quality scores
2024-10-09T06:01:15 prefetch.3.1.1: Current preference is set to retrieve SRA Normalized Format files with full base quality scores
2024-10-09T06:01:15 prefetch.3.1.1: Current preference is set to retrieve SRA Normalized Format files with full base quality scores
2024-10-09T06:01:15 prefetch.3.1.1: Current preference is set to retrieve SR

### STEP 3.5 Converting Multiple SRA files to Fastq


In this step, the SRA files will be processed in parallel using parallel-fastq-dump. Each SRR ID from accs.txt will be read, and xargs will be used to execute parallel-fastq-dump for each SRA ID. This will result in the creation of two paired-end FASTQ files for each SRR ID, which will be compressed into a .gz file to save space.

In [9]:
!cat accs.txt | xargs -P $THREADS -I {} fastq-dump --outdir data/raw_fastq/ --gzip data/raw_fastq/{}/{}.sra

Read 41530740 spots for data/raw_fastq/SRR11550229/SRR11550229.sra
Written 41530740 spots for data/raw_fastq/SRR11550229/SRR11550229.sra
Read 37087937 spots for data/raw_fastq/SRR11550225/SRR11550225.sra
Written 37087937 spots for data/raw_fastq/SRR11550225/SRR11550225.sra
Read 40255612 spots for data/raw_fastq/SRR11550223/SRR11550223.sra
Written 40255612 spots for data/raw_fastq/SRR11550223/SRR11550223.sra
Read 46053552 spots for data/raw_fastq/SRR11550221/SRR11550221.sra
Written 46053552 spots for data/raw_fastq/SRR11550221/SRR11550221.sra
Read 49645505 spots for data/raw_fastq/SRR11550227/SRR11550227.sra
Written 49645505 spots for data/raw_fastq/SRR11550227/SRR11550227.sra
Read 53090577 spots for data/raw_fastq/SRR11550231/SRR11550231.sra
Written 53090577 spots for data/raw_fastq/SRR11550231/SRR11550231.sra


As before, it is good practice to turn .fastq files into .fastq.gz files to save space.

In our case, we will actually need to concatenate the fastq files later on, and so will zip after this.

The no redundant SRA files can also be deleted to save more space.

In [10]:
#find and delete all SRR subfolders in the raw_fastq directory
!find data/raw_fastq -type d -name 'SRR*' -exec rm -rf {} \;

find: ‘data/raw_fastq’: No such file or directory


### STEP 3.6 Download reference transcriptome files that will be used by STAR


This step downloads and prepares the reference data needed for your RNA-seq analysis. It retrieves three essential files:

- Mouse genome (Mus_musculus.GRCm39.dna.primary_assembly.fa.gz): This compressed FASTA file contains the complete mouse genome sequence, that will be used as the reference for aligning your RNA-seq reads.
- Mouse gene annotations (Mus_musculus.GRCm39.104.gtf.gz): This compressed GTF file provides information about the genes and transcripts in the mouse genome, including their locations and structures. This data will crucial for interpreting the aligned RNA-seq reads and understanding what genes are expressed in each.
- Mouse feature table (GCF_000001635.27_GRCm39_feature_table.txt.gz): This compressed table provides additional annotations for the mouse genome features, potentially including information about gene functions and pathways. This step will further used to analyze the differential gene expression (DEG) analysis. 

In [9]:
!wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/002/985/GCF_000002985.6_WBcel235/GCF_000002985.6_WBcel235_genomic.fna.gz -O data/reference/celegans_genome.fa.gz
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/002/985/GCF_000002985.6_WBcel235/GCF_000002985.6_WBcel235_genomic.gtf.gz -O data/reference/celegans_annotation.gtf.gz
!wget -O data/reference/celegans_feature_table.txt.gz https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/002/985/GCF_000002985.6_WBcel235/GCF_000002985.6_WBcel235_feature_table.txt.gz

--2024-10-09 07:11:13--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/002/985/GCF_000002985.6_WBcel235/GCF_000002985.6_WBcel235_genomic.fna.gz
           => ‘data/reference/celegans_genome.fa.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.12, 130.14.250.13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.11|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/all/GCF/000/002/985/GCF_000002985.6_WBcel235 ... done.
==> SIZE GCF_000002985.6_WBcel235_genomic.fna.gz ... 31710738
==> PASV ... done.    ==> RETR GCF_000002985.6_WBcel235_genomic.fna.gz ... done.
Length: 31710738 (30M) (unauthoritative)

GCF_000002985.6_WBc 100%[===================>]  30.24M  7.93MB/s    in 3.8s    

2024-10-09 07:11:18 (7.93 MB/s) - ‘data/reference/celegans_genome.fa.gz’ saved [31710738]

--2024-10-09 07:11:18--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/00

In [ ]:
#!wget https://sra-data-athena.s3.amazonaws.com/reference/celegans_genomic.fna -O data/reference/celegans_genomic.fna
#!wget https://sra-data-athena.s3.amazonaws.com/reference/celegans_genomic.gtf -O data/reference/celegans_annnotation.gtf
#!wget https://sra-data-athena.s3.amazonaws.com/reference/celegans_feature_table.txt -O data/reference/celegans_feature_table.txt

In [10]:
!gunzip -f data/reference/celegans_genome.fa.gz
!gunzip -f data/reference/celegans_annotation.gtf.gz
!gunzip -f data/reference/celegans_feature_table.txt.gz

### STEP 3.7: Copy data file for Trimmomatic

One of trimmomatics functions is to trim sequence machine specific adapter sequences. These are usually within the trimmomatic installation directory in a folder called adapters.

Directories of packages within conda installations can be confusing, so in the case of using conda with trimmomatic, it may be easier to simply download or create a file with the relevant adapter sequencecs and store it in an easy to find directory.

In [12]:
!wget https://sra-data-athena.s3.amazonaws.com/reference/TruSeq3-SE.fa -O data/trimmed/TruSeq3-SE.fa

--2024-10-09 06:30:16--  https://sra-data-athena.s3.amazonaws.com/reference/TruSeq3-SE.fa
Resolving sra-data-athena.s3.amazonaws.com (sra-data-athena.s3.amazonaws.com)... 52.216.138.115, 3.5.29.24, 52.217.140.49, ...
Connecting to sra-data-athena.s3.amazonaws.com (sra-data-athena.s3.amazonaws.com)|52.216.138.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120 [binary/octet-stream]
Saving to: ‘data/trimmed/TruSeq3-SE.fa’

data/trimmed/TruSeq 100%[===================>]     120  --.-KB/s    in 0s      

2024-10-09 06:30:16 (5.20 MB/s) - ‘data/trimmed/TruSeq3-SE.fa’ saved [120/120]



### STEP 4: Run FastQC

FastQC is an invaluable tool that allows you to evaluate whether there are problems with a set of reads. For example, it will provide a report of whether there is any bias in the sequence composition of the reads.

The below code may take a while to run. To make it run faster we can use threads to speed up the process.

In [13]:
!cat accs.txt | xargs -P $THREADS -I {} fastqc "data/raw_fastq/{}.fastq.gz" -o data/fastqc/

application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Started analysis of SRR11550221.fastq.gz
application/gzip
Started analysis of SRR11550229.fastq.gz
Started analysis of SRR11550227.fastq.gz
Started analysis of SRR11550223.fastq.gz
Started analysis of SRR11550225.fastq.gz
Started analysis of SRR11550231.fastq.gz
Approx 5% complete for SRR11550225.fastq.gz
Approx 5% complete for SRR11550223.fastq.gz
Approx 5% complete for SRR11550221.fastq.gz
Approx 5% complete for SRR11550229.fastq.gz
Approx 5% complete for SRR11550227.fastq.gz
Approx 5% complete for SRR11550231.fastq.gz
Approx 10% complete for SRR11550225.fastq.gz
Approx 10% complete for SRR11550229.fastq.gz
Approx 10% complete for SRR11550221.fastq.gz
Approx 10% complete for SRR11550223.fastq.gz
Approx 10% complete for SRR11550227.fastq.gz
Approx 10% complete for SRR11550231.fastq.gz
Approx 15% complete for SRR11550225.fastq.gz
Approx 15% complete for SRR11550229.fastq.gz
Approx 15% complete for SRR1

Fastqc will output the results in HTML format, as below, for all forward and reverse reads.

In [14]:
from IPython.display import IFrame
IFrame(src='./data/fastqc/SRR11550221_fastqc.html', width=800, height=600)

Although its best practice to look over them individually, tools like multiqc allow one to quickly look at a summary of the quality reports of the fastq files.

For instance, the below table shows which warnings, passes, or failures, from each fastqc report. There are other summaries created as well by multiqc.

In [15]:
!multiqc -f data/fastqc/

import pandas as pd
dframe = pd.read_csv("./multiqc_data/multiqc_fastqc.txt", sep='\t')
display(dframe)


/// ]8;id=677208;https://multiqc.info\MultiQC]8;;\ 🔍 v1.25.1

       file_search | Search path: /home/ec2-user/SageMaker/data/fastqc
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 12/12  data/fastqc/SRR11550223_fastqc.zip
            fastqc | Found 6 reports
     write_results | Data        : multiqc_data
     write_results | Report      : multiqc_report.html
           multiqc | MultiQC complete


,Sample,Filename,File type,Encoding,Total Sequences,Total Bases,Sequences flagged as poor quality,Sequence length,%GC,total_deduplicated_percentage,...,basic_statistics,per_base_sequence_quality,per_sequence_quality_scores,per_base_sequence_content,per_sequence_gc_content,per_base_n_content,sequence_length_distribution,sequence_duplication_levels,overrepresented_sequences,adapter_content
0,SRR11550221,SRR11550221.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,46053552.0,3.5 Gbp,0.0,76.0,47.0,19.118246,...,pass,pass,pass,fail,warn,pass,pass,fail,warn,pass
1,SRR11550223,SRR11550223.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,40255612.0,3 Gbp,0.0,76.0,47.0,19.182218,...,pass,pass,pass,fail,warn,pass,pass,fail,warn,pass
2,SRR11550225,SRR11550225.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,37087937.0,2.8 Gbp,0.0,76.0,47.0,22.119323,...,pass,pass,pass,fail,warn,pass,pass,fail,warn,pass
3,SRR11550227,SRR11550227.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,49645505.0,3.7 Gbp,0.0,76.0,47.0,16.958784,...,pass,pass,pass,fail,warn,pass,pass,fail,warn,pass
4,SRR11550229,SRR11550229.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,41530740.0,3.1 Gbp,0.0,76.0,47.0,17.379409,...,pass,pass,pass,fail,warn,pass,pass,fail,warn,pass
5,SRR11550231,SRR11550231.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,53090577.0,4 Gbp,0.0,76.0,46.0,18.771109,...,pass,pass,pass,fail,warn,pass,pass,fail,warn,pass


## STEP 5: Merging our fastq files (Optional if there are multiple SRR per GSM)

If the project used presents the multiple SRAs per GSM we can use Athena to access the SRA metadata and merging the FASTQ files, the code simplifies the subsequent analysis steps and reduces the number of files to process. This can improve efficiency and reduce computational overhead.
In this study this step was not used.

In [16]:
from pyathena import connect
import pandas as pd

# Use the correct argument name: s3_staging_dir
conn = connect(s3_staging_dir='s3://sra-data-athena/', region_name='us-east-1')

query = """
SELECT *
FROM AwsDataCatalog.srametadata.metadata
WHERE bioproject = 'PRJNAXXXXXXX' #Change to the Bioproject number
AND organism = 'Mus musculus'
"""
df = pd.read_sql(
    query, conn
)
df

/tmp/ipykernel_26883/1769049847.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
Failed to execute query.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pyathena/common.py", line 575, in _execute
    query_id = retry_api_call(
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/tenacity/__init__.py", line 475, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/tenacity/__init__.py", line 376, in iter
    result = action(retry_state)
  File "/home/ec2-use

DatabaseError: Execution failed on sql: 
SELECT *
FROM AwsDataCatalog.srametadata.metadata
WHERE bioproject = 'PRJNAXXXXXXX' #Change to the Bioproject number
AND organism = 'Mus musculus'

An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 3:35: mismatched input '#'. Expecting: '%', '*', '+', '-', '.', '/', 'AND', 'AT', 'EXCEPT', 'FETCH', 'GROUP', 'HAVING', 'INTERSECT', 'LIMIT', 'OFFSET', 'OR', 'ORDER', 'UNION', 'WINDOW', '[', '||', <EOF>
unable to rollback

In [ ]:
#import os so we can easily pass strings to shell commands using 'subprocess'
import os
import subprocess

#now get the accession id's and sample id's from the created dataframe
runs = df['acc'].values
samples = list(set(df['acc'].values))

#sort them to be in numerical order
runs.sort()
samples.sort()
samples

In [ ]:
#now iterate through the samples, 
#because there are two SRRs to a run, 
#this means corresponding SRRs indices to an index of a GSM will be
#gsm index *2, and *2+1 
for index, item in enumerate(samples):
    
    #concatenate the two SRRs
    os.system(f"cat data/raw_fastq/{runs[index*2]}_1.fastq data/raw_fastq/{runs[index*2+1]}_1.fastq > data/raw_fastq/{samples[index]}_1.fastq")
    #delete the previous fastq files to save space
    os.system(f"rm data/raw_fastq/{runs[index*2]}_1.fastq")
    os.system(f"rm data/raw_fastq/{runs[index*2+1]}_1.fastq")
    #zip the merged fastq file to save more space
    os.system(f"gzip data/raw_fastq/{samples[index]}_1.fastq")
    
    #repeat for reverse reads
    os.system(f"cat data/raw_fastq/{runs[index*2]}_2.fastq data/raw_fastq/{runs[index*2+1]}_2.fastq > data/raw_fastq/{samples[index]}_2.fastq")
    
    os.system(f"rm data/raw_fastq/{runs[index*2]}_2.fastq")
    os.system(f"rm data/raw_fastq/{runs[index*2+1]}_2.fastq")  
   
    #its good practice to zip files to save space
    os.system(f"gzip data/raw_fastq/{samples[index]}_2.fastq")

In [ ]:
#since our files will now be samples, not SRRs we can write a new text file to use for downstream batch processes.
#we can use the DF we made in the previous cell.
with open('samples.txt', 'w') as f:
    df = df.sort_values(by='sample_name', ascending=True)
    samples = df['acc'].unique()
    samples = '\n'.join(map(str, samples))
    f.write(samples)
    
!cat samples.txt

## STEP 6: Run Trimmomatic

Trimmomatic will trim off any adapter sequences or low quality sequence it detects in the FASTQ files.

Using piping and our original list, it is possible to queue up a batch run of trimmomatic for all our files, note that this is a different way to run a loop compared with what we did before.

The below code may take approximately 30 minutes to run.

In [19]:
!cat accs.txt | xargs -I {} \
trimmomatic SE -threads $THREADS \
'data/raw_fastq/{}.fastq.gz' \
'data/trimmed/{}_trimmed.fastq' \
ILLUMINACLIP:data/trimmed/TruSeq3-SE.fa:2:30:10 LEADING:3 TRAILING:3 MINLEN:36

TrimmomaticSE: Started with arguments:
 -threads 7 data/raw_fastq/SRR11550221.fastq.gz data/trimmed/SRR11550221_trimmed.fastq ILLUMINACLIP:data/trimmed/TruSeq3-SE.fa:2:30:10 LEADING:3 TRAILING:3 MINLEN:36
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTA'
Using Long Clipping Sequence: 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC'
ILLUMINACLIP: Using 0 prefix pairs, 2 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Reads: 46053552 Surviving: 46003886 (99.89%) Dropped: 49666 (0.11%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -threads 7 data/raw_fastq/SRR11550223.fastq.gz data/trimmed/SRR11550223_trimmed.fastq ILLUMINACLIP:data/trimmed/TruSeq3-SE.fa:2:30:10 LEADING:3 TRAILING:3 MINLEN:36
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTA'
Using Long Clipping Sequence: 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC'
ILLUMINACLIP: Using 0 prefix pairs, 2 forward/reverse seq

## STEP 7: Run FastQC
It's best practice to run FastQC after trimming. However, you may decide to run FastQC only once, before or after trimming.

We will proceed with only the forward reads -- this is because, looking at trimmomatic, there were very few 'orphaned' reads. That is to say, most forward and reverse reads were successfully paired together. Because we are just trying to map to a transcriptome, the read lengths of the forward reads alone, in this case, around 60 millions~ basepairs, should be sufficient.

The below code may take around 15-20 minutes to run.

In [5]:
# Run FastQC
!cat accs.txt | xargs -P $THREADS -I {} fastqc data/trimmed/{}_trimmed.fastq -o data/fastqc_samples/

null
null
null
null
null
null
Started analysis of SRR11550231_trimmed.fastq
Started analysis of SRR11550227_trimmed.fastq
Started analysis of SRR11550225_trimmed.fastq
Started analysis of SRR11550221_trimmed.fastq
Started analysis of SRR11550223_trimmed.fastq
Started analysis of SRR11550229_trimmed.fastq
Approx 5% complete for SRR11550225_trimmed.fastq
Approx 5% complete for SRR11550223_trimmed.fastq
Approx 5% complete for SRR11550229_trimmed.fastq
Approx 5% complete for SRR11550221_trimmed.fastq
Approx 5% complete for SRR11550227_trimmed.fastq
Approx 5% complete for SRR11550231_trimmed.fastq
Approx 10% complete for SRR11550225_trimmed.fastq
Approx 10% complete for SRR11550223_trimmed.fastq
Approx 10% complete for SRR11550229_trimmed.fastq
Approx 10% complete for SRR11550221_trimmed.fastq
Approx 10% complete for SRR11550227_trimmed.fastq
Approx 10% complete for SRR11550231_trimmed.fastq
Approx 15% complete for SRR11550225_trimmed.fastq
Approx 15% complete for SRR11550223_trimmed.fastq


## STEP 8: Run MultiQC
MultiQC reads in the FastQC reports and generate a compiled report for all the analyzed FASTQ files.

In [6]:
#!multiqc -f data/fastqc_samples/
!multiqc -f -o data/multiqc_samples/ data/fastqc_samples/


/// ]8;id=537580;https://multiqc.info\MultiQC]8;;\ 🔍 v1.25.1

       file_search | Search path: /home/ec2-user/SageMaker/data/fastqc_samples
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 12/12  lm  
            fastqc | Found 6 reports
     write_results | Data        : data/multiqc_samples/multiqc_data
     write_results | Report      : data/multiqc_samples/multiqc_report.html
           multiqc | MultiQC complete


## STEP 9: Preparing the STAR-Compatible RSEM Reference

This command prepares a reference genome and annotation files for RNA-Seq analysis using RSEM (RNA-Seq by Expectation-Maximization) and STAR (Spliced Transcripts Alignment to a Reference). It generates files needed to quantify gene and isoform expression. The rsem-prepare-reference function takes a GTF file with gene annotations (mouse_annotation.gtf) and a FASTA file with the reference genome sequence (mouse_genome.fa). It processes these files to create a reference, saving the output in the mouse_reference directory. The --star option ensures the reference is compatible with STAR for efficient transcriptome alignment. The -p $THREADS option sets the number of threads used for parallel processing, speeding up the preparation process.

In [13]:
!rsem-prepare-reference --gtf data/reference/celegans_annotation.gtf --star -p $THREADS data/reference/celegans_genome.fa celegans_reference

rsem-extract-reference-transcripts celegans_reference 0 data/reference/celegans_annotation.gtf None 0 data/reference/celegans_genome.fa
Parsed 200000 lines
Parsed 400000 lines
Parsed 600000 lines
Parsing gtf File is done!
data/reference/celegans_genome.fa is processed!
56721 transcripts are extracted and 0 transcripts are omitted.
Extracting sequences is done!
Group File is generated!
Transcript Information File is generated!
Chromosome List File is generated!
Extracted Sequences File is generated!

rsem-preref celegans_reference.transcripts.fa 1 celegans_reference
Refs.makeRefs finished!
Refs.saveRefs finished!
celegans_reference.idx.fa is generated!
celegans_reference.n2g.idx.fa is generated!

STAR  --runThreadN 7  --runMode genomeGenerate  --genomeDir .  --genomeFastaFiles data/reference/celegans_genome.fa  --sjdbGTFfile data/reference/celegans_annotation.gtf  --sjdbOverhang 100  --outFileNamePrefix celegans_reference
	/home/ec2-user/anaconda3/envs/tensorflow2_p310/bin/STAR-avx2 --r

In [14]:
!STAR --runThreadN $THREADS --runMode genomeGenerate --genomeDir . \
--genomeFastaFiles data/reference/celegans_genome.fa \
--sjdbGTFfile data/reference/celegans_annotation.gtf \
--sjdbOverhang 100 \
--genomeSAindexNbases 12 \
--outFileNamePrefix celegans_reference


	/home/ec2-user/anaconda3/envs/tensorflow2_p310/bin/STAR-avx2 --runThreadN 7 --runMode genomeGenerate --genomeDir . --genomeFastaFiles data/reference/celegans_genome.fa --sjdbGTFfile data/reference/celegans_annotation.gtf --sjdbOverhang 100 --genomeSAindexNbases 12 --outFileNamePrefix celegans_reference
	STAR version: 2.7.11b   compiled: 2024-07-03T14:39:20+0000 :/opt/conda/conda-bld/star_1720017372352/work/source
Oct 09 07:55:09 ..... started STAR run
Oct 09 07:55:09 ... starting to generate Genome files
Oct 09 07:55:10 ..... processing annotations GTF
Oct 09 07:55:13 ... starting to sort Suffix Array. This may take a long time...
Oct 09 07:55:14 ... sorting Suffix Array chunks and saving them to disk...
Oct 09 07:55:34 ... loading chunks from disk, packing SA...
Oct 09 07:55:37 ... finished generating suffix array
Oct 09 07:55:37 ... generating Suffix Array index
Oct 09 07:55:47 ... completed Suffix Array index
Oct 09 07:55:47 ..... inserting junctions into the genome indices
Oct 09 

## STEP 10: Run STAR for Alignment, Prepare and Run RSEM for Quantification

This script automates RNA-Seq gene expression quantification using RSEM and STAR. It reads SRR accession IDs from accs.txt, saves results in data/rsem_output, and runs rsem-calculate-expression for each ID. It uses paired-end trimmed FASTQ files from data/trimmed/ and a STAR-aligned RSEM reference (mouse_reference).

In [15]:
import os

# Ensure you've set the path to the RSEM binary
with open('accs.txt', 'r') as f:
    srr_accessions = [line.strip() for line in f.readlines()]

THREADS = 7  # Define number of threads
output_dir = "data/rsem_output"

for srr in srr_accessions:
    os.system(f"rsem-calculate-expression -p {THREADS} --star "
              f"data/trimmed/{srr}_trimmed.fastq celegans_reference "
              f"{output_dir}/{srr}celegans")

STAR --genomeDir .  --outSAMunmapped Within  --outFilterType BySJout  --outSAMattributes NH HI AS NM MD  --outFilterMultimapNmax 20  --outFilterMismatchNmax 999  --outFilterMismatchNoverLmax 0.04  --alignIntronMin 20  --alignIntronMax 1000000  --alignMatesGapMax 1000000  --alignSJoverhangMin 8  --alignSJDBoverhangMin 1  --sjdbScore 1  --runThreadN 7  --genomeLoad NoSharedMemory  --outSAMtype BAM Unsorted  --quantMode TranscriptomeSAM  --outSAMheaderHD \@HD VN:1.4 SO:unsorted  --outFileNamePrefix data/rsem_output/SRR11550221celegans.temp/SRR11550221celegans  --readFilesIn data/trimmed/SRR11550221_trimmed.fastq 
	/home/ec2-user/anaconda3/envs/tensorflow2_p310/bin/STAR-avx2 --genomeDir . --outSAMunmapped Within --outFilterType BySJout --outSAMattributes NH HI AS NM MD --outFilterMultimapNmax 20 --outFilterMismatchNmax 999 --outFilterMismatchNoverLmax 0.04 --alignIntronMin 20 --alignIntronMax 1000000 --alignMatesGapMax 1000000 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 --sjdbScore 1 -

RSEM's indices might be corrupted, unassigned_transcript_572 appears more than once!


"rsem-parse-alignments celegans_reference data/rsem_output/SRR11550221celegans.temp/SRR11550221celegans data/rsem_output/SRR11550221celegans.stat/SRR11550221celegans data/rsem_output/SRR11550221celegans.temp/SRR11550221celegans.bam 1 -tag XM" failed! Plase check if you provide correct parameters/options for the pipeline!
STAR --genomeDir .  --outSAMunmapped Within  --outFilterType BySJout  --outSAMattributes NH HI AS NM MD  --outFilterMultimapNmax 20  --outFilterMismatchNmax 999  --outFilterMismatchNoverLmax 0.04  --alignIntronMin 20  --alignIntronMax 1000000  --alignMatesGapMax 1000000  --alignSJoverhangMin 8  --alignSJDBoverhangMin 1  --sjdbScore 1  --runThreadN 7  --genomeLoad NoSharedMemory  --outSAMtype BAM Unsorted  --quantMode TranscriptomeSAM  --outSAMheaderHD \@HD VN:1.4 SO:unsorted  --outFileNamePrefix data/rsem_output/SRR11550223celegans.temp/SRR11550223celegans  --readFilesIn data/trimmed/SRR11550223_trimmed.fastq 
	/home/ec2-user/anaconda3/envs/tensorflow2_p310/bin/STAR-av

RSEM's indices might be corrupted, unassigned_transcript_572 appears more than once!


"rsem-parse-alignments celegans_reference data/rsem_output/SRR11550223celegans.temp/SRR11550223celegans data/rsem_output/SRR11550223celegans.stat/SRR11550223celegans data/rsem_output/SRR11550223celegans.temp/SRR11550223celegans.bam 1 -tag XM" failed! Plase check if you provide correct parameters/options for the pipeline!
STAR --genomeDir .  --outSAMunmapped Within  --outFilterType BySJout  --outSAMattributes NH HI AS NM MD  --outFilterMultimapNmax 20  --outFilterMismatchNmax 999  --outFilterMismatchNoverLmax 0.04  --alignIntronMin 20  --alignIntronMax 1000000  --alignMatesGapMax 1000000  --alignSJoverhangMin 8  --alignSJDBoverhangMin 1  --sjdbScore 1  --runThreadN 7  --genomeLoad NoSharedMemory  --outSAMtype BAM Unsorted  --quantMode TranscriptomeSAM  --outSAMheaderHD \@HD VN:1.4 SO:unsorted  --outFileNamePrefix data/rsem_output/SRR11550225celegans.temp/SRR11550225celegans  --readFilesIn data/trimmed/SRR11550225_trimmed.fastq 
	/home/ec2-user/anaconda3/envs/tensorflow2_p310/bin/STAR-av

## STEP 11: Report the top 10 most highly expressed genes in the samples

Top 10 most highly expressed genes in each wild-type sample.


In [ ]:
import pandas as pd

# Path to RSEM results directory
rsem_results_dir = 'data/rsem_output'

# Loop through each file in accs.txt
for srr_id in open('accs.txt'):
    srr_id = srr_id.strip()  # Remove newline character
    rsem_result_file = f'{rsem_results_dir}/{srr_id}.genes.results'

    # Load the RSEM results into a Pandas DataFrame
    df = pd.read_csv(rsem_result_file, sep='\t')

    # Sort the DataFrame by TPM values in descending order and get the top 10 genes
    top_10_genes = df.sort_values(by='TPM', ascending=False).head(10)

    # Print the top 10 genes with their TPM values
    print(f"Top 10 Genes by TPM for {srr_id}:")
    print(top_10_genes[['gene_id', 'TPM']])

## STEP 12: Report the expression of ENSMUSG00000064351 for each file

Use `grep` to report the expression in the wild-type sample. The fields in the RSEM `genes.results` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [ ]:
import pandas as pd

# Path to RSEM results directory
rsem_results_dir = 'data/rsem_output'

# Target gene ID
target_gene = 'ENSMUSG00000064351'

# Loop through each file in accs.txt
for srr_id in open('accs.txt'):
    srr_id = srr_id.strip()  # Remove newline character
    rsem_result_file = f'{rsem_results_dir}/{srr_id}.genes.results'

    # Load the RSEM results into a Pandas DataFrame
    df = pd.read_csv(rsem_result_file, sep='\t')

    # Filter for the target gene
    target_gene_data = df[df['gene_id'] == target_gene]

    # Print the target gene's TPM value for the SRR ID
    print(f"TPM for {target_gene} in {srr_id}: {target_gene_data['TPM'].values[0]}")

## STEP 13: Export Read counts to S3 Bucket


The code effectively extracts gene expression data from RSEM output files and stores them in a structured format on an S3 bucket. This data will be accessible for further analysis in Tutorial 2 and Tutorial 3.

In [ ]:
import os
import pandas as pd
import boto3

# Define the path to your RSEM output directory
rsem_output_path = "data/rsem_output"

# Define the S3 bucket and output path
s3_bucket = "sra-data-athena"
s3_output_path = "readcounts/"

# Initialize S3 client
s3_client = boto3.client('s3')

# Get a list of all .genes.results files in the directory
genes_files = [f for f in os.listdir(rsem_output_path) if f.endswith('.genes.results')]

# Loop through each file to extract gene ID, expected counts, and gene length
for file in genes_files:
    file_path = os.path.join(rsem_output_path, file)
    
    # Read the .genes.results file
    rsem_data = pd.read_csv(file_path, sep="\t")

    # Check if the necessary columns exist
    if all(col in rsem_data.columns for col in ["gene_id", "expected_count", "length"]):
        # Create a new dataframe with required columns
        result_data = rsem_data[["gene_id", "expected_count", "length"]]
        result_data.columns = ["GeneID", "Count", "GeneLength"]

        # Define the output filename based on the input file name
        output_file_name = f"{os.path.splitext(file)[0]}.txt"
        s3_output_file_path = f"{s3_output_path}{output_file_name}"

        # Convert the DataFrame to a CSV string
        csv_buffer = result_data.to_csv(sep="\t", index=False)

        # Upload the result directly to S3
        s3_client.put_object(Bucket=s3_bucket, Key=s3_output_file_path, Body=csv_buffer)

    else:
        print(f"Warning: Required columns are missing in file: {file}")

# Optionally, print a message indicating completion
print("Extraction and file creation complete.")



## STEP 14: Save Merged Read Counts

This code combines multiple RSEM gene count files into a single, unified file, making it easier to analyze and visualize the gene expression data. This files was also uploaded to S3 Bucket to allow further analysis in other Tutorials. 

In [ ]:
# Ensure the RSEM quantification results directory exists
!mkdir -p data/rsem_output

# Merge RSEM results by gene counts (similar to Salmon's numreads merge)
!rsem-generate-data-matrix data/rsem_output/*.genes.results > data/rsem_output/merged_gene_counts.txt

# Optionally, rename the columns based on the samples
# If you want to assign your GSM identifiers or any other custom names, edit the header.
!sed -i "1s/.*/Name\tGSM6658439\tGSM6658438\tGSM6658435\tGSM6658441\tGSM6658433\tGSM6658431\tGSM6658429\tGSM6658427/" data/rsem_output/merged_gene_counts.txt

# Remove any unnecessary prefixes like 'gene-' or 'rna-' for easier formatting
!sed -i "s/gene-//g" data/rsem_output/merged_gene_counts.txt
!sed -i "s/rna-//g" data/rsem_output/merged_gene_counts.txt

# Show a preview of the merged quantification file
!head data/rsem_output/merged_gene_counts.txt

import boto3
import os

# Define the file path and S3 bucket details
file_path = "data/rsem_output/merged_gene_counts.txt"
bucket_name = "sra-data-athena"
s3_key = "readcounts/merged_gene_counts.txt"

# Initialize an S3 client
s3_client = boto3.client('s3')

# Upload the file to the specified S3 bucket
try:
    s3_client.upload_file(file_path, bucket_name, s3_key)
    print(f"File {file_path} uploaded successfully to {bucket_name}/{s3_key}")
except Exception as e:
    print(f"Error uploading file: {e}")

# Define the file paths and S3 bucket details
rsem_output_path = "data/rsem_output"
feature_table_path = "data/reference/celegans_feature_table.txt"
bucket_name = "sra-data-athena"
s3_output_path = "readcounts/"
s3_feature_table_path = "reference/celegans_feature_table.txt"

# ... (rest of the code remains the same)

# Upload the gene count file
s3_client.upload_file(file_path, bucket_name, s3_key)

# Upload the feature table file
s3_client.upload_file(feature_table_path, bucket_name, s3_feature_table_path)

## STEP 15: Save RSEM reference and STAR index to S3 Bucket

In [ ]:
!aws s3 cp data/rsem_reference s3://sra-data-athena/reference/rsem_reference/ --recursive

## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as using snakemake.


[Workflow One:](Tutorial_1_subsampling_mouse-miniforge.ipynb) A short introduction to downloading and mapping sequences to a mouse genome using STAR and RSEM.


[Workflow Two (DEG Analysis):](Tutorial_2_DEG_Analysis_mouse.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.

[Workflow Three (Network Analysis):](Tutorial_3_NetAct.ipynb) Using NetAct and R to conduct transcription factor network analysis.
